In [14]:
import numpy as np
import pandas as pd
from math import trunc,radians,sin,cos,sqrt,atan2


In [2]:
############## Calculo de distancias####################
def distancia_lat_long_km(filial,pedido): # algoritmo de calculo da distancia feito pela Myllene
    # converte graus em radianos
    lat1, lon1, lat2, lon2 = map(radians, [filial[0], filial[1], pedido[0], pedido[1]])
    
    # raio da Terra em km
    R = 6371
    
    # diferença de latitude e longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # fórmula Haversine
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))
    distancia = R*c
    
    return distancia

In [23]:
########## Função generica de calculo da distancia total do dia #########

def calcular(filiais,clientes):
    atribuicao = {}
    for item in range(len(filiais)):
        atribuicao[item] = []
    somatorio = []
    for pedido in clientes :
        distancia_min = 100000  # distancia iniciada em um valor alto, para garantir a troca no primeiro loop
        i = -1 #variavel gambiarra pra achar o index do array numpy
        for filial in filiais:
            dist = distancia_lat_long_km(filial,pedido)
            dist += distancia_lat_long_km(df_armazens[0],filial) # acrescenta a distancia da filial ao armazem
            i += 1
            if(dist < distancia_min): # troca somente se a distancia total for menor que a anterior
                distancia_min = dist
                salva_filial = i            
        atribuicao[salva_filial].append(pedido)
        somatorio.append(distancia_min)
    resultado = 0
    for i in somatorio:
        resultado += i
    return atribuicao,resultado        
    

In [4]:
df = pd.read_excel('C:/Users/Gabri/Documents/database.xlsx')

In [5]:
############## Inputs#######################

df_datas = df[['DATA_PED']].copy()
df_datas.head(5)
print(type(df_datas.DATA_PED[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [7]:
df_filiais = pd.read_excel('C:/Users/gabri/OneDrive/Documentos/Martins/filiais.xlsx')

In [8]:
df_armazens = pd.read_excel('C:/Users/gabri/OneDrive/Documentos/Martins/armazens.xlsx')

In [9]:
df_clientes = df[['VLRLNTCLI_VND','VLRLTTCLI_VND']].copy()

In [10]:
df_clientes = df_clientes.values
df_armazens = df_armazens.values
df_filiais = df_filiais.values


In [11]:
print(df_armazens)
print(df_clientes)
print(df_filiais)

[[-38.324 -12.698]]
[[-39.5463 -18.0776]
 [-39.5463 -18.0776]
 [-39.5463 -18.0776]
 ...
 [-39.0093 -10.506 ]
 [-39.0093 -10.506 ]
 [-38.7539  -8.8461]]
[[-38.238       -9.4048    ]
 [-40.1808     -10.4658    ]
 [-40.8393     -14.8496    ]
 [-39.1642     -14.4716    ]
 [-42.7676476  -14.21233244]
 [-44.99       -12.153     ]
 [-38.324      -12.698     ]
 [-40.307      -12.528     ]
 [-38.5126     -12.9706    ]
 [-41.8568     -11.2999    ]]


In [24]:
j = 0
final = []
pedidos_por_filiais = []
fim_do_dia = 0
for _ in range(92):
    Clientes_dia = []
    for data in df_datas.DATA_PED[j:]:
        Clientes_dia.append(df_clientes[j])
        if(j == len(df_clientes)-1):
            j += 1
            break
        if(data != df_datas.DATA_PED[j+1]):
            #Clientes_dia = df_clientes[fim_do_dia:(j+1)] # se necessario usar fatiamento arrumar essa linha
            j += 1
            fim_do_dia = j
            break
        j += 1
    pedido_filial , soma = calcular(df_filiais,Clientes_dia)
    final.append(trunc(soma))
    pedidos_por_filiais.append(pedido_filial)


In [25]:
print(final)
print()
print(len(final))

[460910, 130066, 459594, 360160, 231496, 27404, 530860, 607091, 606941, 535495, 397890, 215845, 17232, 462711, 186472, 489660, 508144, 417037, 308587, 37099, 489247, 554399, 485134, 387202, 398865, 243909, 39487, 399333, 457738, 504554, 446603, 306141, 202887, 24523, 394602, 526541, 538682, 490502, 344140, 213345, 26728, 481231, 531201, 509796, 511202, 404870, 168470, 24487, 417494, 454406, 444370, 368704, 296277, 135151, 17736, 353888, 424605, 411966, 381309, 258125, 129430, 13661, 373763, 492930, 527739, 481524, 427964, 240153, 23161, 477957, 563878, 564788, 504898, 381411, 217014, 21712, 435161, 509553, 470336, 434413, 306611, 197946, 16948, 423126, 486692, 471647, 502191, 334699, 189794, 25773, 436996, 15739]

92


In [31]:
res = np.array(final)
fit = np.sqrt(np.mean((res - np.mean(res))**2))
print(f"SMSE = {fit:.1f} km")

SMSE = 176032.6 km
